# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
import sqlalchemy as db
from sqlalchemy.schema import CreateTable
import logging
from sqlalchemy import Column, Integer, String, Float, DateTime, Date
from sqlalchemy import func
from sqlalchemy.orm import sessionmaker

In [2]:
# Constants we need for convenience
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
Here we calculate the distance using haversine formula and we adjust the distance in the unit of kilometers.

In [3]:
def calculate_distance(from_coord, to_coord):
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord
    R = 6373.0
    dlon = radians(lon2) - radians(lon1)
    dlat = radians(lat2) - radians(lat1)
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

Here we test if the calculate distance function works, and it works!

In [4]:
def test_calculate_distance():
    distance = calculate_distance((50,60), (60,60))
    assert distance == 1111

In [5]:
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(lambda dataframe: calculate_distance((dataframe['pickup_latitude'],dataframe['pickup_longitude']), (dataframe['dropoff_latitude'],dataframe['dropoff_longitude'])), axis = 1)

We are testing if the add_distance_column function is working, and it works!

In [6]:
df_test = pd.DataFrame(np.array([[50, 60, 60,60],[50,55,60,55]]),
                   columns=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude','dropoff_longitude'])
add_distance_column(df_test)
df_test

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,distance
0,50,60,60,60,1112.298332
1,50,55,60,55,1112.298332


### Processing Taxi Data

We parse the url to get the link for our csv file. We also apply some constraints to make sure we get the data in the given time frame.

In [7]:
def find_taxi_csv_urls():
    content = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(content.text, 'html.parser')
    
    matches = []
    # find all links on the website
    x = [a['href'] for a in soup.find_all('a')]
    
    # filter for yellow trip data from 2009-01 to 2015-06
    pattern = "yellow_tripdata"
    time_range = ["2009", "2010", "2011", "2012", "2013", "2014","2015-01","2015-02","2015-03","2015-04","2015-05","2015-06"]
    
    for time in time_range:
        for i in range(len(x)):
            if time in x[i] and pattern in x[i]:
                matches.append(x[i])
                
    return matches

In [8]:
find_taxi_csv_urls()

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-03.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-04.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-05.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-06.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-07.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-10.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-11.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-12.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-01.csv',
 'https://s3.amazonaws.co

#### Get and clean the yellow taxi data for each month. 

1. `Preset column names` 

We preset the column names as those files contains relatively similar elements and it also saves us the work to rename columns afterwards. However, we noticed that data for 2015 has one more columns than other years so there is one more column for 2015 data. 

2. `Drop unrelated columns`

We only leave columns that are related to questions in other parts to save program run time. We also filtered out the rows in which the coordinates are out of box coordinates

In [9]:
def get_and_clean_month_taxi_data(url):
    
    col_names_2015 = ["vendor_id", 
                      "pickup_datetime", 
                      "dropoff_datetime",
                      "passenger_count", 
                      "trip_distance", 
                      "pickup_longitude", 
                      "pickup_latitude", 
                      "rate_code", 
                      "store_and_fwd_flag", 
                      "dropoff_longitude", 
                      "dropoff_latitude", 
                      "payment_type",
                      "fare_amount",
                      "surcharge",
                      "mta_tax",
                      "tip_amount",
                      "tolls_amount",
                      "improvement_surcharge",
                      "total_amount"]
    
    col_names_else = ["vendor_id", 
                      "pickup_datetime", 
                      "dropoff_datetime",
                      "passenger_count", 
                      "trip_distance", 
                      "pickup_longitude", 
                      "pickup_latitude", 
                      "rate_code", 
                      "store_and_fwd_flag", 
                      "dropoff_longitude", 
                      "dropoff_latitude", 
                      "payment_type",
                      "fare_amount",
                      "surcharge",
                      "mta_tax",
                      "tip_amount",
                      "tolls_amount",
                      "total_amount"]
    
    
    # drop useless columns and rename some of the columns
    # year 2015 has different column name so if statement is used here
    if "2015" in url:
        df_taxi = pd.read_csv(url, header=None, names =col_names_2015, low_memory=False)
        df_taxi = df_taxi.iloc[1:,:]
        df_taxi.drop(["vendor_id", 
                      "dropoff_datetime", 
                      "passenger_count", 
                      "trip_distance", 
                      "rate_code", 
                      "store_and_fwd_flag", 
                      "payment_type", 
                      "fare_amount", 
                      "surcharge", 
                      "mta_tax", 
                      "tolls_amount", 
                      "improvement_surcharge", 
                      "total_amount"], inplace=True, axis=1)
    else:
        df_taxi = pd.read_csv(url, header=None, names =col_names_else, low_memory=False)
        df_taxi = df_taxi.iloc[1:,:]
        df_taxi.drop(["vendor_id", 
                      "dropoff_datetime", 
                      "passenger_count", 
                      "trip_distance", 
                      "rate_code", 
                      "store_and_fwd_flag", 
                      "payment_type", 
                      "fare_amount", 
                      "surcharge", 
                      "mta_tax", 
                      "tolls_amount", 
                      "total_amount"], inplace=True, axis=1)
                
    # clean data that's outside of the BOX COORDS
    df_taxi['pickup_longitude'] = df_taxi['pickup_longitude'].astype(float)
    df_taxi['dropoff_longitude'] = df_taxi['dropoff_longitude'].astype(float)
    df_taxi['dropoff_latitude'] = df_taxi['dropoff_latitude'].astype(float)
    df_taxi['pickup_latitude'] = df_taxi['pickup_latitude'].astype(float)
    
    df_taxi=df_taxi[(df_taxi['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])&(df_taxi['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1])]
    df_taxi=df_taxi[(df_taxi['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])&(df_taxi['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1])]
    df_taxi=df_taxi[(df_taxi['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])&(df_taxi['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0])]
    df_taxi=df_taxi[(df_taxi['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])&(df_taxi['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0])]
    
    return df_taxi

In [10]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    all_csv_urls = find_taxi_csv_urls()
    
    for csv_url in all_csv_urls:
        
        dataframe = get_and_clean_month_taxi_data(csv_url)
        
        # sample the same size from the yellow taxi data set as the uber data each month in the time range
        month = csv_url[-11:-4]
        dataframe = dataframe.sample(len(uber_data[uber_data['pickup_datetime'].str.contains(month)]))
        
        # add distance column
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes, axis=0)
    return taxi_data

### Processing Uber Data

We drop unrelated columns and filtered out the rows in which the coordinates are out of box coordinates.

In [11]:
def load_and_clean_uber_data(csv_file):
    df_uber = pd.read_csv(csv_file)
    
    # drop useless columns and rename some of the columns
    df_uber.drop(["Unnamed: 0", "key", "fare_amount", "passenger_count"], inplace=True, axis="columns")
        
    # clean data that's outside of the BOX COORDS
    df_uber=df_uber[(df_uber['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])&(df_uber['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1])]
    df_uber=df_uber[(df_uber['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])&(df_uber['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1])]
    df_uber=df_uber[(df_uber['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])&(df_uber['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0])]
    df_uber=df_uber[(df_uber['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])&(df_uber['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0])]
    
    return df_uber

### Get Uber data and a quick glance

In [12]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    # add the distance column
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [13]:
uber_data = get_uber_data()
uber_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1.683993
1,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,2.458493
2,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,5.893465
3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,1.837688
4,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5.656533
...,...,...,...,...,...,...
199995,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,0.118212
199996,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,2.448332
199997,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,15.727903
199998,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,3.676230


### Get and sample yellow taxi data and a quick glance

In [45]:
# set seed so we have same result every time we run the code
np.random.seed(123)
taxi_data = get_and_clean_taxi_data()
taxi_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,distance
484715,2009-01-10 12:55:00,-73.994825,40.739978,-74.006367,40.730120,0,1.682893
13187615,2009-01-30 21:59:12,-73.988559,40.670395,-73.899596,40.770505,0,14.841043
13334276,2009-01-24 13:15:41,-73.967479,40.763241,-73.996558,40.716687,0,6.096219
534040,2009-01-06 18:14:57,-73.969972,40.759178,-73.984404,40.748630,0,1.983424
13850826,2009-01-08 08:41:48,-73.984937,40.739625,-73.990305,40.735101,0,0.778412
...,...,...,...,...,...,...,...
11263406,2015-06-28 12:48:42,-73.966370,40.764778,-73.974724,40.753372,1.45,1.570752
11234583,2015-06-16 13:19:26,-73.954063,40.770321,-73.952515,40.778206,1.36,0.893730
10539330,2015-06-15 21:52:01,-73.988182,40.746029,-74.007637,40.740807,8.06,2.230690
4431434,2015-06-11 21:37:28,-73.993500,40.761288,-73.951378,40.785927,3.56,5.418497


In [48]:
# reindex the dataframe to look cleaner
taxi_data = taxi_data.reset_index(drop=True)
taxi_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,distance
0,2009-01-10 12:55:00,-73.994825,40.739978,-74.006367,40.730120,0,1.682893
1,2009-01-30 21:59:12,-73.988559,40.670395,-73.899596,40.770505,0,14.841043
2,2009-01-24 13:15:41,-73.967479,40.763241,-73.996558,40.716687,0,6.096219
3,2009-01-06 18:14:57,-73.969972,40.759178,-73.984404,40.748630,0,1.983424
4,2009-01-08 08:41:48,-73.984937,40.739625,-73.990305,40.735101,0,0.778412
...,...,...,...,...,...,...,...
195467,2015-06-28 12:48:42,-73.966370,40.764778,-73.974724,40.753372,1.45,1.570752
195468,2015-06-16 13:19:26,-73.954063,40.770321,-73.952515,40.778206,1.36,0.893730
195469,2015-06-15 21:52:01,-73.988182,40.746029,-74.007637,40.740807,8.06,2.230690
195470,2015-06-11 21:37:28,-73.993500,40.761288,-73.951378,40.785927,3.56,5.418497


In [49]:
# save file to local path for convenient use for other questions
taxi_data.to_csv(r'D:\Project\final-project-4501\taxi_data.csv', header=True)

Read sampled yellow taxi data from local path

In [14]:
taxi_data = pd.read_csv(r'D:\Project\final-project-4501\taxi_data.csv')
taxi_data = taxi_data.iloc[:,1:]
taxi_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,distance
0,2009-01-10 12:55:00,-73.994825,40.739978,-74.006367,40.730120,0.00,1.682893
1,2009-01-30 21:59:12,-73.988559,40.670395,-73.899596,40.770505,0.00,14.841043
2,2009-01-24 13:15:41,-73.967479,40.763241,-73.996558,40.716687,0.00,6.096219
3,2009-01-06 18:14:57,-73.969972,40.759178,-73.984404,40.748630,0.00,1.983424
4,2009-01-08 08:41:48,-73.984937,40.739625,-73.990305,40.735101,0.00,0.778412
...,...,...,...,...,...,...,...
195467,2015-06-28 12:48:42,-73.966370,40.764778,-73.974724,40.753372,1.45,1.570752
195468,2015-06-16 13:19:26,-73.954063,40.770321,-73.952515,40.778206,1.36,0.893730
195469,2015-06-15 21:52:01,-73.988182,40.746029,-74.007637,40.740807,8.06,2.230690
195470,2015-06-11 21:37:28,-73.993500,40.761288,-73.951378,40.785927,3.56,5.418497


### Processing Weather Data

We save only three columns `['DATE','HourlyWindSpeed','HourlyPrecipitation']` for weather data because that's what we need.

In [15]:
def clean_month_weather_data_hourly(csv_file):
    hourly_data = pd.read_csv(csv_file,low_memory=False)
    # fill unreported blank cells with 0
    hourly_data=hourly_data[['DATE','HourlyWindSpeed','HourlyPrecipitation']].fillna('0')
    hourly_data['DATE'] = pd.to_datetime(hourly_data['DATE'])
    hourly_data['HourlyPrecipitation']=hourly_data.apply (lambda row: convert(row), axis=1)
    hourly_data['HourlyPrecipitation'] = hourly_data['HourlyPrecipitation'].astype(float)
    hourly_data['HourlyWindSpeed'] = hourly_data['HourlyWindSpeed'].astype(float)
    return hourly_data
    

We defined a function to clean and refill the `HourlyPrecipitation` column

In [16]:
def convert(dataframe):
    if 'T' in dataframe.HourlyPrecipitation:
        return float(0)
    elif 's' in dataframe.HourlyPrecipitation:
        return dataframe.HourlyPrecipitation[:-1]
    else:
        return dataframe.HourlyPrecipitation

We defined a function to convert the `Hourly Data` to `Daily Data`

In [17]:
def clean_month_weather_data_daily(csv_file):
    hourly_data = clean_month_weather_data_hourly(csv_file)
    daily_data = hourly_data.groupby([hourly_data['DATE'].dt.date]).sum()/24
    daily_data.columns=['DailyWindSpeed','DailyPrecipitation']
    return daily_data

We defined two functions to process all weather data files and generate hourly data and daily data

In [18]:
def load_and_clean_weather_data_hourly():
    hourly_dataframes = []
    
    weather_csv_files = ["2009_weather.csv",
                         "2010_weather.csv",
                         "2011_weather.csv",
                         "2012_weather.csv",
                         "2013_weather.csv",
                         "2014_weather.csv",
                         "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes, axis=0)
    hourly_data.reset_index(inplace = True)
    return hourly_data

In [19]:
def load_and_clean_weather_data_daily():
    daily_dataframes = []
    
    weather_csv_files = ["2009_weather.csv",
                         "2010_weather.csv",
                         "2011_weather.csv",
                         "2012_weather.csv",
                         "2013_weather.csv",
                         "2014_weather.csv",
                         "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        daily_dataframes.append(daily_dataframe)
        
    daily_data = pd.concat(daily_dataframes, axis=0)
    daily_data.reset_index(inplace = True)
    return daily_data

### Get weather data and a quick view

In [20]:
data_hourly = load_and_clean_weather_data_hourly()
data_hourly

,index,DATE,HourlyWindSpeed,HourlyPrecipitation
0,0,2009-01-01 00:51:00,18.0,0.0
1,1,2009-01-01 01:51:00,18.0,0.0
2,2,2009-01-01 02:51:00,18.0,0.0
3,3,2009-01-01 03:51:00,8.0,0.0
4,4,2009-01-01 04:51:00,11.0,0.0
...,...,...,...,...
77967,11382,2015-12-31 21:51:00,0.0,0.0
77968,11383,2015-12-31 22:51:00,7.0,0.0
77969,11384,2015-12-31 23:51:00,5.0,0.0
77970,11385,2015-12-31 23:59:00,0.0,0.0


In [21]:
data_daily = load_and_clean_weather_data_daily()
data_daily

,DATE,DailyWindSpeed,DailyPrecipitation
0,2009-01-01,11.041667,0.000000
1,2009-01-02,8.791667,0.000000
2,2009-01-03,9.875000,0.000000
3,2009-01-04,8.291667,0.000000
4,2009-01-05,7.791667,0.000000
...,...,...,...
2546,2015-12-27,9.208333,0.007083
2547,2015-12-28,8.208333,0.001250
2548,2015-12-29,15.250000,0.038750
2549,2015-12-30,6.625000,0.012083


## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [22]:
engine = db.create_engine(DATABASE_URL, echo=True)

In [23]:
sqllogger = logging.getLogger("sqlalchemy.engine.Engine")
formatter = logging.Formatter("[sqlalchemy] %(message)s")
sqllogger.handlers[0].setFormatter(formatter)

In [24]:
Base = declarative_base()

In [25]:
class Taxi(Base):
    __tablename__ = "taxi_data" 
    
    id = Column(Integer, primary_key=True)
    pickup_datetime  = Column(DateTime)
    dropoff_datetime = Column(DateTime)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    tip_amount = Column(Float)
    total_amount = Column(Float)
    distance = Column(Float)     

class Uber(Base):
    __tablename__ = "uber"
    
    id = Column(Integer, primary_key=True)
    pickup_datetime  = Column(DateTime)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    distance = Column(Float) 

class Hourly_Data(Base):
    __tablename__ = "hourly_weather"
    
    id = Column(Integer, primary_key=True)
    Date  = Column(DateTime)
    HourlyWindSpeed = Column(Float)
    HourlyPrecipitation = Column(Float)

class Daily_Data(Base):
    __tablename__ = "daily_weather"
    
    id = Column(Integer, primary_key=True)
    Date  = Column(Date)
    DailyWindSpeed = Column(Float)
    DailyPrecipitation = Column(Float)

In [26]:
Base.metadata.create_all(engine)

[sqlalchemy] BEGIN (implicit)
[sqlalchemy] PRAGMA main.table_info("taxi_data")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("uber")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("hourly_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("daily_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] COMMIT


In [27]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = str(CreateTable(Taxi.__table__))

DAILY_WEATHER_SCHEMA = str(CreateTable(Daily_Data.__table__))

TAXI_TRIPS_SCHEMA = str(CreateTable(Hourly_Data.__table__))

UBER_TRIPS_SCHEMA = str(CreateTable(Uber.__table__))

In [28]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [31]:
taxi_data.to_sql('yellow_taxi', engine, if_exists='append', index=False)
uber_data.to_sql('uber', engine, if_exists='append',index=False)
data_daily.to_sql('daily_weather', engine, if_exists='append', index=False)
data_hourly.to_sql('hourly_weather', engine, if_exists='append',index=False)
#if_exists='append'

[sqlalchemy] PRAGMA main.table_info("yellow_taxi")
[sqlalchemy] [raw sql] ()
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] INSERT INTO yellow_taxi (pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, tip_amount, distance) VALUES (?, ?, ?, ?, ?, ?, ?)
[sqlalchemy] [generated in 0.62109s] (('2009-01-10 12:55:00', -73.994825, 40.739978, -74.006367, 40.73012, 0.0, 1.6828933955740737), ('2009-01-30 21:59:12', -73.988559, 40.670395, -73.899596, 40.770505, 0.0, 14.84104344013526), ('2009-01-24 13:15:41', -73.967479, 40.763241, -73.996558, 40.716687, 0.0, 6.0962194791545885), ('2009-01-06 18:14:57', -73.969972, 40.759178, -73.984404, 40.74863, 0.0, 1.9834241246838145), ('2009-01-08 08:41:48', -73.984937, 40.739625, -73.990305, 40.735101, 0.0, 0.7784118768516332), ('2009-01-24 19:21:31', -74.010623, 40.703442, -74.007369, 40.707143, 0.0, 0.5459663898343566), ('2009-01-21 20:30:05', -74.006673, 40.706364, -74.005535, 40.73092, 1.95, 2.7342473742194264), ('2009-0

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [86]:
q_most_popular_hour_taxi = """
SELECT strftime('%H', pickup_datetime) as hour,count(*)
FROM yellow_taxi 
GROUP BY hour 
ORDER BY count(*) DESC
"""

In [87]:
engine.execute(q_most_popular_hour).fetchall()

[sqlalchemy] 
SELECT strftime('%H', pickup_datetime) as hour,count(*)
FROM yellow_taxi GROUP BY hour order by count(*) DESC

[sqlalchemy] [raw sql] ()


[('19', 61540),
 ('18', 58850),
 ('20', 57330),
 ('21', 56460),
 ('22', 54090),
 ('14', 49415),
 ('23', 49115),
 ('17', 48845),
 ('12', 47830),
 ('15', 46930),
 ('13', 46920),
 ('09', 46205),
 ('11', 45980),
 ('08', 44060),
 ('10', 43695),
 ('16', 40465),
 ('00', 38810),
 ('07', 34850),
 ('01', 28485),
 ('02', 21200),
 ('06', 19960),
 ('03', 15235),
 ('04', 11645),
 ('05', 9445)]

In [76]:
print(engine.execute(q_most_popular_hour_taxi).fetchall()[0][0], 'was the most popular to take a yellow taxi.')

[sqlalchemy] 
SELECT strftime('%H', pickup_datetime) as hour,count(*)
FROM yellow_taxi GROUP BY hour order by count(*) DESC

[sqlalchemy] [raw sql] ()
19 was the most popular to take a yellow taxi.


In [77]:
with open('Most_Popular_Hour_Taxi.sql', "w") as f:
    f.write(q_most_popular_hour)

* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.

In [102]:
q_most_popular_day_uber = """
SELECT strftime('%w', pickup_datetime) AS day, COUNT(*)
FROM yellow_taxi 
GROUP BY day
ORDER BY COUNT(*) DESC
"""

In [103]:
engine.execute(q_most_popular_day_uber).fetchall()

[sqlalchemy] 
SELECT strftime('%w', pickup_datetime) AS day, COUNT(*)
FROM yellow_taxi 
GROUP BY day
ORDER BY COUNT(*) DESC

[sqlalchemy] [raw sql] ()


[('5', 149005),
 ('6', 148305),
 ('4', 147190),
 ('3', 142950),
 ('2', 137870),
 ('0', 127925),
 ('1', 124115)]

In [104]:
print('Friday was the most popular to take an Uber.')

Friday was the most popular to take an Uber.


In [105]:
with open('Most_Popular_Day_Uber.sql', "w") as f:
    f.write(q_most_popular_day_uber)

* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?

In [86]:
q_95_distance = """
SELECT distance
FROM
(
SELECT pickup_datetime, distance FROM yellow_taxi
WHERE pickup_datetime LIKE '2013-07%'
UNION
SELECT pickup_datetime, distance FROM uber
WHERE pickup_datetime LIKE '2013-07%'
ORDER BY distance ASC
)
"""

In [87]:
engine.execute(q_95_distance).fetchall()

[sqlalchemy] 
SELECT distance
FROM
(
SELECT pickup_datetime, distance FROM yellow_taxi
WHERE pickup_datetime LIKE '2013-07%'
UNION
SELECT pickup_datetime, distance FROM uber
WHERE pickup_datetime LIKE '2013-07%'
ORDER BY distance ASC
)

[sqlalchemy] [raw sql] ()


[(0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0001112298329618,),
 (0.00011122983296185665,),
 (0.0009562454342731,),
 (0.0009562454342731319,),
 (0.0017891930620298636,),
 (0.003241562703765,),
 (0.00324156270376504,),
 (0.0033516784373125445,),
 (0.0066108288161783,),
 (0.006610828816178349,),
 (0.0066787190151616,),
 (0.006678719015161673,),
 (0.0093853878341393,),
 (0.00938538783413937,),
 (0.009774418425032,),
 (0.009774418425032058,),
 (0.0175314152707801,),
 (0.017531415270780143,),
 (0.022662548708972687,),
 (0.035920053600352116,),
 (0.0424970984186075,),
 (0.0453790355980589,),
 (0.04537903559805893,),
 (0.05345967398824787,),
 (0.0551368180869453,),
 (0.055136818086945365,),
 (0.0628299500173872,),
 (0.06282995001738721,),
 (0.07363437540441052,),
 (0.07690511712856647,),
 (0.0832811780194857,),
 (0.08328117801948574,),
 (0.093203

* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?

* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?

* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)